# Plots for differentiation and thermal evolution<a class="anchor" id="top"></a>
[Summary statistics](#stats)
For the whole thermal evolution:
+ [temperature](#temp)
+ [% silicate melted](#silicate)
+ [Rayleigh number](#critical-rayleigh)
+ [Stagnant lid thickness](#d0)
+ [Combined lid thickness and Ra](#Ra-bl)
+ [Urey ratio](#Urey)


For differentiation stage:
+ [% iron melted](#iron)

For the thermal evolution stage:
+ [temperatures and heat fluxes](#temp-heat)
+ [inner core size and magnetic Reynolds number](#Rem)
+ [magnetic field strength](#B)
+ [dynamo duration times](#duration)

## Set-up

In [ ]:
#import modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import sys
sys.path.append('../')
from load_info import load_run_info, load_run_results

#scale time to Myr
from plotting_constants import Myr, Tml, Tms

Choose run and whether you want to save figure

In [ ]:
run=1
save = False# do you want to save your figures?
automated = False #if this run was one of an automated group
path = '../Results_combined/' #path to files
#path = '../Results_combined/rcmf_run2/' #path to files
#path = '../Run_params/berlin_test4/'
save_path = '../Plots/'
#save_path = '../Plots/Paper_run4/'

Load npz files for differentiation

In [ ]:
npzfile = np.load(f'{path}run_{run}_diff.npz')
Tdiff = npzfile['Tdiff'] 
iron = npzfile['Xfe']
silicate_d = npzfile['Xsi']
tdiff = npzfile['t_diff']
Ra_d = npzfile['Ra']
Ra_crit_d = npzfile['Ra_crit']
d0_diff = npzfile['d0']

Load npz files for thermal evolution

In [ ]:
npzfile = np.load(f'{path}run_{run}.npz')
Tc= npzfile['Tc'] 
Tc_conv = npzfile['Tc_conv']
Tcmb = npzfile['Tcmb']
Tm_mid = npzfile['Tm_mid']
Tm_conv = npzfile['Tm_conv']
Tm_surf = npzfile['Tm_surf'] 
T_profile = npzfile['T_profile']
f = npzfile['f'] 

t = npzfile['t'] #time in s
Rem = npzfile['Rem'] # magnetic Reynolds number 
B = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Flux = npzfile['Flux']
Ra = npzfile['Ra'] 
RaH = npzfile['RaH'] 
RanoH = npzfile['RanoH'] 
Racrit = npzfile['Racrit'] 
d0 = npzfile['d0'] 
min_unstable = npzfile['min_unstable'] 
Urey = npzfile['Ur']
Xs = npzfile['Xs']
dl = npzfile['dl']
dc = npzfile['dc']
Fs = Flux[0]
Fcmb = Flux[1]
Fad = Flux[2]
Frad = Flux[3]

Run Info

In [ ]:
if automated == True:
    r, dr, tstart, dt, viscosity = load_run_info(run,f'{path}auto_params.csv')
else: 
    r, dr, tstart, dt, viscosity = load_run_info(run,f'{path}run_info.csv')

Calculate silicate melt content for thermal evolution

In [ ]:
nmantle = int((r/dr)/2)
silicate_t = (T_profile[:,nmantle:] - Tms)/(Tml-Tms)

Concatenate shared variables

In [ ]:
Tall = np.hstack((Tdiff,np.transpose(T_profile)))
tall = np.append(tdiff,t)
iron_all = np.hstack
Ra_all = np.append(Ra_d,Ra)
Ra_crit_all = np.append(Ra_crit_d,Racrit)
d0_all = np.append(d0_diff,d0)

Scale time, choose whether or not to plot logarithmically

In [ ]:
t_plot_all = tall/Myr
t_plot_t = t/Myr
log_time = True # do you want to plot time logarithmically

## Summary statistics <a class="anchor" id="stats"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
results = load_run_results(run,f'{path}run_results.csv')

In [ ]:
terode = results.at[0,'terode']
tstrat_remove = results.at[0,'tstrat_remove']
diff_time = results.at[0,'diff_time']

In [ ]:
print(f"Differentiation is at {results.at[0,'diff_time']:.2f} Myr")
print(f"The temperature at differentiation is at {results.at[0,'diff_T']:.2f}K")
print(f"Peak magma ocean temp is {results.at[0,'peakT']:.0f}K at {results.at[0,'tmax']:.2f} Myr")
print(f"Stratification starts at {results.at[0,'tstrat_start']:.2f} Myr")
print(f"Mantle hotter than the core until {results.at[0,'tstrat_remove']:.2f} Myr")
print(f"Erosion of stratification by {results.at[0,'terode']:.2f} Myr") 
print(f"End of mantle convection by {results.at[0,'fcond_t']:.2f} Myr")
print(f"The maximum thermal Rem is {results.at[0,'max_Rtherm']:.2f} at {results.at[0,'max_Rtherm']:.2f} Myr")
print(f"The maximum thermal field strength is {results.at[0,'max_Btherm']:.2e} T at {results.at[0,'max_Bthermt']:.2f} Myr")
print(f"The maximum compositional Rem is {results.at[0,'max_Rcomp']:.2f}")
print(f"The maximum compositional field strength is {results.at[0,'max_Bcomp']:.2e}")

In [ ]:
peak_coreT = np.amax(Tall[:nmantle,:])
loc_max = np.where(Tall[:nmantle,:]==peak_coreT)[1][0] #take the set of time coordinates and first value (they should all be the same)
tcoremax = tall[loc_max]/Myr
fcond_t = results.at[0,'fcond_t']
diff_T =results.at[0,'diff_T']

In [ ]:
print(f'The peak core temperature is {peak_coreT:.0f}K at {tcoremax:.1f}Myr')

On and off times for the dynamo

In [ ]:
var_results = pd.read_csv(path+'run_results.csv',skiprows=[1])
on1=var_results.loc[var_results['run']==run,'magon_1'].values[0]
off1=var_results.loc[var_results['run']==run,'magoff_1'].values[0]
on2=var_results.loc[var_results['run']==run,'magon_2'].values[0]
off2=var_results.loc[var_results['run']==run,'magoff_2'].values[0]
on3=var_results.loc[var_results['run']==run,'magon_3'].values[0]
off3=var_results.loc[var_results['run']==run,'magoff_3'].values[0]

## Temperature <a class="anchor" id="temp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
rplot = np.arange(0,int(r)+dr,int(dr))/1e3

In [ ]:
r_unstable=np.array([]) 
for ind in min_unstable:
    r_unstable = np.append(r_unstable,rplot[int(ind)])
rc = r/2

In [ ]:
plt.figure(rasterized=True,figsize=[15,10])
plt.pcolormesh(t_plot_all[::2],rplot[::2],Tall[::2,::2],shading = 'gouraud',vmin=200,vmax=1600)
#plt.pcolormesh(t_plot_all,rplot,Tall,shading = 'gouraud',vmin=200,vmax=1650)
plt.hlines(rc/1e3,t_plot_t[0],max(t_plot_all),linestyle='--',color='black',label='CMB')
plt.vlines(t_plot_t[0],0,r/1e3,linestyle='-.',label='Differentiation')
plt.fill_betweenx([0,rc/5e3],on1,off1,alpha=0,hatch='/',label='dynamo on')
plt.plot(t_plot_t,r_unstable,linestyle='dotted',label='Convecting core')
if on2 > 0:
    plt.fill_betweenx([0,rc/5e3],on2,off2,alpha=0,hatch='/')
if on3 > 0: 
    plt.fill_betweenx([0,rc/5e3],on3,off3,alpha=0,hatch='/')

if np.any(t_plot_t<fcond_t):
    plt.plot(t_plot_t[(t_plot_t<=fcond_t)&(d0<(r-rc))],(r-d0[(t_plot_t<=fcond_t)&(d0<(r-rc))])/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
    plt.plot(t_plot_t[(t_plot_t<=fcond_t)&(r_unstable==0)][1:],(rc+dl[(t_plot_t<=fcond_t)&(r_unstable==0)][1:])/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
    plt.vlines(t_plot_t[t_plot_t<=fcond_t][-1],r/1e3,rc/1e3,linestyle='dotted',label='conductive mantle',color='red')
else:
    plt.plot(t_plot_t,(r-d0)/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
    #plt.plot(t_plot_t[r_unstable==0][1:],(rc+dl)[r_unstable==0][1:]/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
    plt.plot(t_plot_t,(rc+dl)/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
#plt.vlines(t[f<0.999000][0]/Myr,0/1e3,rc/1e3,linestyle='dotted',label='beginning of core solidification',color='black')
#labels and limits
plt.ylabel('r /km')
plt.xlabel('t / Myr')
plt.colorbar(label='T/K')
plt.title(f"Run {run}")
plt.xscale('log')
plt.legend(bbox_to_anchor=[1.4,0.5])
if save == True:
    plt.savefig(f'{save_path}thermal_profile_{run}.png',bbox_inches='tight')

Single temperature profile (choose the time of interest)

In [ ]:
plt.scatter(t_plot_t,Tcmb-T_profile[:,nmantle+1],label='CMB-mantle temp')
plt.xlabel('Time/Myr')
plt.ylabel('Tcmb-Tm/K')

In [ ]:
plt.scatter(t_plot_t[:5],Tcmb[:5],label='CMB')
plt.scatter(t_plot_t[:5],T_profile[:5,nmantle-1],label='core')
plt.scatter(t_plot_t[:5],T_profile[:5,nmantle+1],label='mantle')
plt.legend()

In [ ]:
colors = ['black','darkblue','blue','royalblue','cornflowerblue','paleturquoise','salmon','maroon','saddlebrown','chocolate','orange','tan','olive','olivedrab','yellowgreen','darkolivegreen']
for i in range(3):
    plt.scatter(T_profile[i,:],rplot,label=f'{0.01*(i+1)} post diff',s=5,color=colors[i])
plt.scatter(Tdiff[:,-1],rplot,label='pre diff',s=5)
plt.ylabel('Distance from centre/km')
plt.xlabel('Temperature/K')
plt.legend(loc='lower left')
#plt.ylim([rc/1e3,r/1e3])
plt.xlim([1570,1582])

## Melted silicate content <a class="anchor" id="silicate"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
plt.pcolormesh(tdiff/Myr, rplot,silicate_d,shading = 'gouraud',vmin=0,vmax=0.2) # use same vmin and vmax as other plots to make them comparable
#plt.yscale('log')
plt.ylabel('r /km')
plt.xlabel('t / Myr')
#plt.ylim([0,400])
plt.title(f'Run {run} - during differentiation')
plt.colorbar(label='proportion of silicate melted')
if save ==True:
    plt.savefig(f'{save_path}si_diff_run_{run}.png')

In [ ]:
rmantle = np.arange(int(r/2),int(r+dr),int(dr))/1e3
plt.figure()
plt.pcolormesh(t/Myr, rmantle,np.transpose(silicate_t),shading = 'gouraud',vmin=0, vmax=1) # use same vmin and vmax as other plots to make them comparable
plt.xscale('log')
plt.ylabel('r /km')
plt.xlabel('t / Myr')
#plt.ylim([0,400])
plt.title(f'Run {run} - after differentiation')
plt.colorbar(label='proportion of silicate melted')
if save ==True:
    plt.savefig(f'{save_path}si_therm_run_{run}.png')

## Rayleigh number<a class="anchor" id="critical-rayleigh"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Blue scatter points are where Ra is less than 50% of critical. Orange points are where it is more than 110% of critical.

In [ ]:
plt.figure()
plt.plot(tdiff/Myr,Ra_d,color='navy')
plt.plot(t[t_plot_t<fcond_t]/Myr,RaH[t_plot_t<fcond_t],label='$Ra_H$',color='seagreen')
plt.plot(t[t_plot_t<fcond_t]/Myr,RanoH[t_plot_t<fcond_t],label='Ra',color='cornflowerblue')
plt.plot(t_plot_all[t_plot_all<fcond_t],Ra_crit_all[t_plot_all<fcond_t],label='$Ra_c$',linestyle='dashed',color='black') 
plt.plot(t_plot_all[t_plot_all<fcond_t],Ra_all[t_plot_all<fcond_t],label='model Ra',color='navy') 
plt.scatter(t_plot_all[Ra_all/Ra_crit_all<0.5],Ra_all[Ra_all/Ra_crit_all<0.5])
plt.scatter(t_plot_all[Ra_all/Ra_crit_all>1.1],Ra_all[Ra_all/Ra_crit_all>1.1])
plt.ylim(bottom=1e0)#,top=1e8)
#plt.xlim([min(tall/Myr),5])
#plt.vlines(tdiff[-1]/Myr,1e3,1e18,linestyle='dotted',color='blue',label='differentiation')
#if conduction == True:
    #plt.vlines(cond_t,1e3,1e18,linestyle='dashed',color='k',label='conduction')
    #plt.xlim([min(tall/Myr),cond_t])
plt.ylabel('Ra')
#plt.xscale('log')
plt.yscale('log')
plt.xlabel('t / Myr')
#plt.xlim([0,100])
plt.ylim([1e4,1e12])
plt.title(f'Run {run}')
plt.legend()
#if save == True:
#plt.savefig(f'{save_path}Ra_run{run}.png')

In [ ]:
plt.figure(tight_layout=True)
plt.subplot(2,1,1)
plt.semilogy(t_plot_all[d0_all<r],Ra_crit_all[d0_all<r],label='Ra crit') 
plt.semilogy(t_plot_all[d0_all<r],Ra_all[d0_all<r],label='Ra') 
plt.ylim(bottom=1e0)
#plt.xlim([min(tall/Myr),30])
plt.vlines(tdiff[-1]/Myr,1e3,1e25,linestyle='dotted',color='blue',label='differentiation')
plt.ylabel('Ra')
plt.xscale('log')
plt.xlabel('t / Myr')
plt.legend()
plt.subplot(2,1,2)
plt.plot(t_plot_all[d0_all<r],(Tall[int(nmantle/2),d0_all<r]-1400)/400)
plt.vlines(tdiff[-1]/Myr,0,0.8,linestyle='dotted',color='blue',label='differentiation')
#plt.xlim([min(tall/Myr),30])
plt.ylim([0.1,0.8])
plt.xscale('log')
plt.ylabel('Mid mantle silicate melt fraction')
plt.suptitle(f'Run {run} - Mantle convecting')
plt.xlabel('t / Myr')
#if save == True:
    #plt.savefig(f'{save_path}Ra_phi_comp_run{run}.png',dpi=300)

## Stagnant lid thickness <a class="anchor" id="d0"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
fcond_t=15

In [ ]:
plt.figure()
plt.scatter(tall[(tall/Myr)<fcond_t]/Myr,d0_all[(tall/Myr)<fcond_t]/(r-rc),label='$\delta_0$') 
#plt.ylim(top=1e0)
#plt.xlim([min(tall/Myr),25])
plt.vlines(tdiff[-1]/Myr,0,1,linestyle='dotted',color='blue',label='differentiation')
plt.hlines(1,min(tall/Myr),max(tall/Myr),linestyle='-.',color='green',label='mantle thickness')
plt.ylabel('$\delta_0$/Mantle thickness')
plt.xlabel('t / Myr')
plt.xscale('log')
plt.yscale('log')
plt.title(f'Run {run}')
plt.legend()
if save == True:
    plt.savefig(f'{save_path}d0_run{run}.png')

In [ ]:
nlid_cells = np.round(d0/dr)
lid_start = nmantle - nlid_cells.astype(int) - 1
plt.scatter(t_plot_t[d0/rc<1],lid_start[d0/rc<1])

In [ ]:
lid_start = nmantle - nlid_cells.astype(int) - 1
Tplot = np.zeros([len(t_plot_t)])
for i,lid in enumerate(lid_start):
    Tplot[i] = T_profile[i,lid-1]
plt.scatter(t_plot_t[:5],Tplot[:5])

### CMB boundary layer thickness

In [ ]:
plt.figure()
plt.plot(t_plot_t[(t_plot_t<fcond_t)&(min_unstable ==0)],dl[(t_plot_t<fcond_t)&(min_unstable ==0)]/rc,label='$\delta_l$') 
#plt.ylim(top=1e0)
#plt.xlim([min(tall/Myr),25])
plt.hlines(1,min(tall/Myr),max(tall/Myr),linestyle='-.',color='green',label='mantle thickness')
plt.hlines(dr/rc,min(tall/Myr),max(tall/Myr),linestyle='-.',color='black',label='cell thickness')
plt.ylabel('$\delta_l$/Mantle thickness')
plt.xlabel('t / Myr')
plt.xscale('log')
plt.yscale('log')
plt.title(f'Run {run}')
plt.legend()
if save == True:
    plt.savefig(f'{save_path}dl_run{run}.png')

### How does viscosity relate to lid thickness?

In [ ]:
from viscosity_def import viscosity

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,sharex=True)
ax2 = ax.twinx()
ax.scatter(t_plot_t[t_plot_t<fcond_t],viscosity(((Tm_conv+Tcmb)/2)[t_plot_t<fcond_t],'vary'),color='black',label='$\\eta$ ($T_m$+$T_{cmb}$)/2')
ax.scatter(t_plot_t[t_plot_t<fcond_t],viscosity(Tm_conv[t_plot_t<fcond_t],'vary'),color='green',label='$T_m$')
ax2.plot(t_plot_t[t_plot_t<fcond_t],dl[t_plot_t<fcond_t]/dr,linestyle='dashed')
ax2.hlines(rc/dr,xmin=0,xmax=fcond_t,label='mantle thickness')
ax.set_xlabel('Time/Myr')
ax.set_xscale('log')
ax2.set_ylabel('CMB boundary layer/cell thickness')
ax.set_ylabel('Viscosity/Pas')
ax.legend()
ax2.legend()
ax.set_yscale('log')
#ax2.set_yscale('log')

### Temperature difference between mantle and CMB and lid thickness

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,sharex=True)
ax.plot(t_plot_t[t_plot_t<fcond_t],(T_profile[:,nmantle+1]-Tcmb)[t_plot_t<fcond_t],label='$T_m$-$T_{CMB}$')
ax2 = ax.twinx()
ax2.plot(t_plot_t[t_plot_t<fcond_t],dl[t_plot_t<fcond_t]/dr,color='green',linestyle='dashed',label='lid thickness')
ax.set_xlabel('Time/Myr')
ax.set_ylabel('Temperature difference /K')
ax2.set_ylabel('CMB b.l thickness/mantle thickness')
ax.legend()
#ax.set_ylim([-0.1,0.1])
ax2.set_ylim([-1,2])
ax.set_xscale('log')
#ax.set_xlim([1,5])

Temperature difference between CMB, mantle and surface

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,sharex=True)
ax.scatter(t_plot_t[t_plot_t<fcond_t],(T_profile[:,-1]-T_profile[:,nmantle])[t_plot_t<fcond_t],label='$T_s$-$T_{CMB}$')
ax.scatter(t_plot_t[t_plot_t<fcond_t],(T_profile[:,-1]-T_profile[:,nmantle+1])[t_plot_t<fcond_t],label='$T_s$-$T_m$')
ax.scatter(t_plot_t[t_plot_t<fcond_t],(T_profile[:,nmantle+1]-T_profile[:,nmantle])[t_plot_t<fcond_t]-1315,label='$T_m$-$T_{CMB}$ -1315')
ax2 = ax.twinx()
#ax2.scatter(t_plot_t[t_plot_t<fcond_t],dl[t_plot_t<fcond_t]/rc,color='green',linestyle='dashed')
ax.set_xlabel('Time/Myr')
ax.set_ylabel('Temperature difference /K')
ax2.set_ylabel('CMB b.l thickness/mantle thickness')
ax.legend()
ax.set_xlim([0,20])
#ax.set_ylim([-1325,-1315])
#ax.set_ylim([-0.5,0.1])
#ax.set_xscale('log')

## Combined b.l thickness and Ra <a class="anchor" id="Ra-bl"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.plot(t_plot_t,dl/rc)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,sharex='col',tight_layout=True)
ax2 = ax.twinx()
# boundary layer thickness
ax.plot(t_plot_t[t_plot_t<fcond_t],dl[t_plot_t<fcond_t]/1e3,label='$\delta_l$',color='seagreen',linestyle='dashed') 
ax.plot(t_plot_t[t_plot_t<fcond_t],rc/1e3-d0[t_plot_t<fcond_t]/1e3,label='$\delta_0$',color='cornflowerblue',linestyle='dashed',alpha=0.5)
# Rayleigh number
ax2.plot(t_plot_t,Ra,label='Rayleigh number',color='red')
ax2.plot(t_plot_t,Racrit,label='Critical Rayleigh number',color='black')

ax.set_ylabel('Height in the mantle /km')
ax.set_ylim([0,rc/1e3])
ax.set_xlabel('Time /Myr')
ax2.set_ylabel('Rayleigh number')
ax2.set_yscale('log')
ax.set_xscale('log')
ax.legend(loc='center left')
ax2.legend()
if save == True:
    plt.savefig(f'{save_path}criterion_comp_run{run}.png')

Time difference between subcritical Ra and lids being thicker than domain.

In [ ]:
time_diff = fcond_t - t_plot_t[Ra<Racrit][0]
print(f"There is a {time_diff:.1f} Myr time difference between subcritical Ra and lids being thicker than domain")

## Urey ratio  <a class="anchor" id="Urey"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
plt.plot(t_plot_t,Urey) 
#plt.xlim([min(tall/Myr),25])
plt.hlines(1,min(tall/Myr),max(tall/Myr),linestyle='-.',color='black')
if conduction == True:
    plt.vlines(cond_t,0,1,linestyle='dashed',color='k',label='conduction')
    plt.legend()
plt.ylabel('Urey ratio')
plt.xlabel('t / Myr')
plt.xscale('log')
plt.ylim([0,1.1])
plt.title(f'Run {run}')
if save == True:
    plt.savefig(f'{save_path}Urey{run}.png')

## Melted iron content during differentiation<a class="anchor" id="iron"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
plt.pcolormesh(tdiff/Myr, rplot,iron,shading = 'gouraud')#,vmin=0,vmax=1) # use same vmin and vmax as other plots to make them comparable
#plt.yscale('log')
plt.ylabel('r /km')
plt.xlabel('t / Myr')
#plt.ylim([290,300])
plt.title(f'Run {run}')
plt.colorbar(label='proportion of iron melted')
if save ==True:
    plt.savefig(f'{save_path}fe_diff_run_{run}.png')

# Temperature and Flux Plot <a class="anchor" id="temp-heat"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure(tight_layout=True,figsize=[10,7])
#plt.suptitle(f'Thermal evolution of a {r/1e3:.0f}km asteroid \n run {run} viscosity {viscosity}')

xmin=tstart

#temperatures as function of time
plt.subplot(2,1,1)

plt.plot(t_plot_t,Tc,label='central T$_c$')
plt.plot(t_plot_t[Tc_conv!=0],Tc_conv[Tc_conv!=0],label='convective T$_c$')
plt.plot(t_plot_t,Tcmb,label='T$_{cmb}$')
plt.plot(t_plot_t,Tm_mid,label='T$_m$ - middle')
plt.plot(t_plot_t[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective T$_m$')
#plt.scatter(t_plot_t,T_profile[:,-400],label='T one above CMB')
#plt.scatter(t_plot_t,T_profile[:,-399],label='T two above CMB')

if log_time == True:
    plt.xscale('log')
#if conduction == True:
plt.vlines(fcond_t,ymin=min(Tm_surf),ymax=1600,color='black',linestyle='--',label='conduction')
plt.ylim([1400,1600]) #use these limits when comparing runs
#plt.ylim([1520,1530])
plt.ylabel('T/K')
#plt.legend(loc='lower left', ncol= 2)
plt.legend(loc='lower right', ncol= 2)
#fluxes as function of time
plt.subplot(2,1,2)

Fcmb_neg = Fcmb[Fcmb<0]
Fcmb_pos = Fcmb[Fcmb>0]

plt.semilogy(t_plot_t,Fs,label='$F_s$')
plt.scatter(t_plot_t[Fcmb<0],abs(Fcmb_neg),label='$-F_{CMB}$',color='navy',s=2)
plt.scatter(t_plot_t[Fcmb>0],Fcmb_pos,label='$F_{CMB}$',color='mediumorchid',s=2)
plt.semilogy(t_plot_t,Fad,label='$F_{ad}$')
plt.semilogy(t_plot_t,Frad,label='$F_{rad}$')
plt.fill_betweenx(y=[1e-4,0.1],x1=tstrat_remove,x2=terode,label='erosion of stratification',alpha=0.2)
if log_time == True:
    plt.xscale('log')
plt.xlabel('Time/ Myr')

plt.ylim([1e-4,1e2])   #use these limits when comparing runs
plt.ylabel('Flux/ W$m^{-2}$')
plt.legend(loc='upper right',ncol=2)

if save == True:
    plt.savefig(f'{save_path}run_{run}_Tflux.png',dpi=450)


# Magnetic Field Strength <a class="anchor" id="B"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Extract buoyancy fluxes for core solidification and thermal dynamo.

In [ ]:
import pandas as pd
from parameters import f0

In [ ]:
comp = buoyr[0,:]
therm = buoyr[1,:]

In [ ]:
compdf = pd.Series(comp[f<f0])
thermdf = pd.Series(therm[f<f0])
Remdf = pd.Series(Rem[f<f0])
Bdf = pd.Series(B[f<f0])

### New plot

Test a range of window widths for rolling average

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',tight_layout=True,figsize=[8,5])

ln1 = ax[0].plot(t_plot_t,comp)
ax[1].plot(t_plot_t,therm)
for i in range(1,6,1):
    ax[0].plot(t_plot_t[f<f0],compdf.rolling(window=10*i,center=True).mean(),label=f'{10*i}')
    ax[1].plot(t_plot_t[f<f0],thermdf.rolling(window=10*i,center=True).mean(),label=f'{10*i}')
#ax[0].legend(ln1+ln2,['Compositional','Thermal'])
ax[0].set_ylabel('Compositional buoyancy flux/ kg$s^{-1}$')
#ax2.set_yscale('log')
ax[1].set_ylabel('Thermal buoyancy flux/ kg$s^{-1}$')
ax[0].legend()
ax[1].legend()

A window width of 30--50 for data after the onset of core solidification seems sufficient to smooth the data.

In [ ]:
from parameters import Xs_eutectic

### B, Rem and buoyancy flux with time

In [ ]:
wn = 50
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',figsize=[10,8])
#B and Rem
ax2 = ax[0].twinx()
ln1 = ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color='black')
ax[0].plot(t_plot_t[f<f0],Bdf.rolling(window=wn,center=True).mean(),color='black')
ln2 = ax2.plot(t_plot_t[f>=f0],Rem[f>=f0],color='darkorchid')
ax2.plot(t_plot_t[f<f0],Remdf.rolling(window=wn,center=True).mean(),color='darkorchid')
ax2.hlines(10,min(t_plot_t),max(t_plot_t),linestyle='dashed',color='grey')
ax[0].legend(ln1+ln2,['B','Re$_m$'])
ax[0].set_ylabel('Magnetic field strength/ $\mu T$')
ax2.set_ylabel('$Re_m$')

#thermal and compositional buoyancy flux
ax3 = ax[1].twinx()
ln3 = ax[1].plot(t_plot_t[f>=f0],therm[f>=f0],color='black')
ax[1].plot(t_plot_t[f<f0],thermdf.rolling(window=wn,center=True).mean(),color='black')
ln4 = ax[1].plot(t_plot_t[f>=f0],comp[f>=f0],color='darkorchid')
ax[1].plot(t_plot_t[f<f0],compdf.rolling(window=wn,center=True).mean(),color='darkorchid')
ln5 = ax3.plot(t_plot_t[(f<f0)&(Xs<Xs_eutectic)],(compdf.rolling(window=wn,center=True).mean()/thermdf.rolling(window=wn,center=True).mean())
         [Xs[f<f0]<Xs_eutectic])
ax[1].set_xlabel('Time/Ma')
ax[1].legend(ln3+ln4+ln5,['thermal','compositional','ratio'])
ax[1].set_ylabel('Buoyancy flux /kg$s^{-1}$')
ax[1].set_yscale('log')
ax3.set_ylabel('Compositional/Thermal buoyancy flux')
ax3.set_ylim([0,20])

## Magnetic Reynolds number and inner core size <a class="anchor" id="Rem"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure(tight_layout=True,figsize=[8,6])
plt.suptitle(f'Thermal evolution of a {r/1e3:.0f}km asteroid \n core size')

plt.subplot(2,1,1)
plt.plot(t_plot_t,f,label='f')
plt.xscale('log')
plt.xlabel('Time/ Myr')
plt.ylabel('f')

plt.subplot(2,1,2)
plt.plot(t_plot_t,Xs,label='Xs')
plt.xscale('log')
plt.xlabel('Time/ Myr')
plt.ylabel('Xs /wt%')


if save == True:
    plt.savefig(f'{save_path}run_{run}_Remf.png',dpi=450)

## Dynamo duration <a class="anchor" id="duration"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
from duration_calc import on_off_load

In [ ]:
print(f'The dynamo starts at {on1:.2f} Myr, stops at {off1:.2f} Myr and lasts {off1-on1:.2f} Myr')
if on2 > 0:
    print(f'The second dynamo starts at {on2:.2f} Myr, stops at {off2:.2f} Myr and lasts {off2-on2:.2f} Myr')
if on3 > 0:
    print(f'The third dynamo starts at {on3:.2f} Myr, stops at {off3:.2f} Myr and lasts {off3-on3:.2f} Myr')